In [ ]:
import os
import io
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import torchvision
import pandas as pd
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
label_mappings = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6
}


class GuitarDataset(Dataset):
    """Guitar Dataset"""

    def __init__(self, root_dir, transform=None, train=True):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        if train:
          self.root_dir = os.path.join(root_dir, 'train')
        else:
          self.root_dir = os.path.join(root_dir, 'test')
        self.transform = transform
        self.image_names = [name for name in os.listdir(self.root_dir) if name.endswith('.jpg') or name.endswith('.jpeg')]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        """
        Funzione per ottenere un elemento del dataset

        :param idx: Indice dell'elemento
        :return: Una tupla (immagine, label)
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.image_names[idx])
        image = cv.imread(img_name)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image = np.moveaxis(image, 2, 0)
        image = image.astype(np.float32) / 255.0
        image = torch.from_numpy(image)

        img_base_name = os.path.basename(img_name)
        label = label_mappings.get(img_base_name.split(' ')[0])

        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [ ]:
# montare Google Drive. Si può fare dall'interfaccia grafica del sito

In [ ]:
num_classes = 7

In [ ]:
model = torchvision.models.mobilenet_v2(pretrained=True)
model

MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

# Uncomment the following line to train only the last layers
#set_parameter_requires_grad(model, feature_extract)

In [ ]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [ ]:
test_classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.2, inplace=False),
    torch.nn.Linear(in_features=1280, out_features=1280),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=1280, out_features=num_classes),
)

In [ ]:
model.classifier = test_classifier
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1280, bias=True)
  (2): ReLU()
  (3): Linear(in_features=1280, out_features=7, bias=True)
)

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
        transforms.Resize((280, 218))
])

#dataset_str = 'cropped_images'
#dataset_str = 'cropped_rotated_images'
#dataset_str = 'cropped_processed_rotated_images'
#dataset_str = 'cropped_rotated_processed_images_1'
dataset_str = 'cropped_rotated_processed_images_2'

training_dataset = GuitarDataset('drive/MyDrive/chords_data/' + dataset_str, transform=transform, train=True)

train_loader = DataLoader(training_dataset, batch_size=32, shuffle=True, num_workers=8)

testing_dataset = GuitarDataset('drive/MyDrive/chords_data/' + dataset_str, transform=transform, train=False)

validation_loader = DataLoader(testing_dataset, batch_size=32, shuffle=True, num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f"GPU found! Using {device}...")

num_epochs = 20
learning_rate = 0.001

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

GPU found! Using cuda:0...


In [ ]:
model = model.to(device)

In [ ]:
best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    print('-' * 10)
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
            for i, (imgs, labels) in enumerate(tqdm(train_loader)):
              labels = torch.nn.functional.one_hot(labels, num_classes=7).float()
              imgs = imgs.to(device)
              labels = labels.to(device)
              outputs = model(imgs)
              #import pdb; pdb.set_trace()
              outputs = torch.nn.functional.softmax(outputs, dim=1)
              loss = criterion(outputs, labels)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

        else:
            model.eval()   # Set model to evaluate mode
            num_correct = 0
            num_samples = 0

            with torch.no_grad():
                for x, y in validation_loader:
                    x = x.to(device=device)
                    y = y.to(device=device)

                    scores = model(x)
                    # predictions = torch.tensor([1.0 if i >= 0.5 else 0.0 for i in scores]).to(device)
                    predictions = scores.argmax(1)
                    num_correct += (predictions == y).sum()
                    num_samples += predictions.size(0)
                    acc = float(num_correct) / float(num_samples) * 100
                    print(
                        f"Got {num_correct} / {num_samples} with accuracy {acc:.2f}"
                    )
                if acc > best_acc:
                  best_acc = acc
                  best_model_wts = copy.deepcopy(model.state_dict())

----------
Epoch 0/19


  0%|          | 0/18 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Got 15 / 32 with accuracy 46.88
Got 35 / 64 with accuracy 54.69
Got 54 / 96 with accuracy 56.25
Got 73 / 128 with accuracy 57.03
Got 73 / 130 with accuracy 56.15
----------
Epoch 1/19


  0%|          | 0/18 [00:00<?, ?it/s]

...

Got 24 / 32 with accuracy 75.00
Got 50 / 64 with accuracy 78.12
Got 75 / 96 with accuracy 78.12
Got 101 / 128 with accuracy 78.91
Got 103 / 130 with accuracy 79.23
----------
Epoch 18/19


  0%|          | 0/18 [00:00<?, ?it/s]

Got 24 / 32 with accuracy 75.00
Got 51 / 64 with accuracy 79.69
Got 76 / 96 with accuracy 79.17
Got 104 / 128 with accuracy 81.25
Got 106 / 130 with accuracy 81.54
----------
Epoch 19/19


  0%|          | 0/18 [00:00<?, ?it/s]

Got 29 / 32 with accuracy 90.62
Got 59 / 64 with accuracy 92.19
Got 84 / 96 with accuracy 87.50
Got 113 / 128 with accuracy 88.28
Got 115 / 130 with accuracy 88.46


In [ ]:
best_acc

89.23076923076924

In [ ]:
torch.save(best_model_wts, 'drive/MyDrive/salvataggi_pytorch/classification_network/' + dataset_str + '_model_state_dict')